In [1]:
import numpy as np
import networkx as nx

#%matplotlib notebook
import matplotlib.pyplot as plt
from matplotlib import cm
from mpl_toolkits.axes_grid1 import make_axes_locatable
%config InlineBackend.figure_format = 'retina'

import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
from torch.optim.lr_scheduler import StepLR
import torch.nn.functional as F
from torch.autograd import Variable
from torch.utils.data.dataset import Dataset
from torch.utils.data.dataset import TensorDataset
from torch.utils.data import DataLoader # (testset, batch_size=4,shuffle=False, num_workers=4)
from torch.optim.lr_scheduler import ReduceLROnPlateau as RLRP
from torch.nn.parallel import DistributedDataParallel, DataParallel
from torch.nn.init import xavier_normal
from torch.nn.parameter import Parameter
import torchvision.datasets as datasets
import torchvision.transforms as transforms

import sys
from datetime import datetime
from functools import reduce
import os
import os.path
import pandas as pd
import pickle
import importlib
from collections import Counter
from copy import deepcopy
from collections import OrderedDict

import torch_geometric as tg
import nkmodel as nk
import ppo.core as core
from ppo.ppo import PPOBuffer
from utils.utils import max_mean_clustering_network
import envs
import json
from itertools import product
from functools import reduce  

In [2]:
E = 32
M = 100
N = 15
K = 7
NN = 3
exp = 8
trj_len = 200
graph_type = 'maxmc'
reward_type = 'indv_raw_full'
action_type = 'total'
extra_type = 'SI'
env_name = 'SL_NK_' + action_type

nx_dict = {'complete': nx.complete_graph, 'ba': nx.barabasi_albert_graph, 'er': nx.erdos_renyi_graph, 'maxmc':max_mean_clustering_network} 
nx_arg_dict = {'complete': {'n': M}, 'ba': {'n': M, 'm': 19}, 'er': {'n': M, 'p': 0.3}, 'maxmc': {'n': M}}

env_kwargs = {
        'E': E,
        'M': M,
        'N': N,
        'K': K,
        'neighbor_num': NN,
        'exp': exp,
        'graph': nx_dict[graph_type],
        'graph_dict': nx_arg_dict[graph_type],
        'reward_type': reward_type,
        'action_type': action_type,
        'extra_type': extra_type,
    'corr_type': 'TT'
    }

## Baseline Test

In [3]:
baseline_data_dict = {}
baseline_data_dict['keys'] = ['Ret', 'FinalScore']

In [ ]:
env_num = 10
test_ensemble_num = 50
env_list = [envs.__dict__[env_name](**env_kwargs) for i in range(env_num)]
state_list = []
for i in range(env_num):
    _, fixed_state = env_list[i].reset(E=test_ensemble_num, base=True)
    state_list.append(deepcopy(fixed_state))
print("Baseline construction initiated")

In [ ]:
baselines = ['FollowBest', 'FollowBest_indv', 'FollowMajor', 'FollowMajor_indv', 'IndvLearning', 'RandomCopy']
#baselines = ['FollowMajor', 'FollowMajor_indv']
for baseline_name in baselines:
    print(f"Baseline : {baseline_name}")
    baseline_data = {}
    baseline_data['Ret'] = []
    baseline_data['FinalScore'] = []
    baseline_data['scr_buf'] = []
    baseline_data['unq_buf'] = []
    
    for i in range(env_num):
        print(i)
        env_base = env_list[i]
        ac_base = core.__dict__[baseline_name](env_base, action_type, extra_type, corr_type='TT')
        scr_buf = np.zeros((test_ensemble_num, M, trj_len), dtype=np.float32)
        unq_buf = np.zeros((test_ensemble_num, trj_len), dtype=np.float32)
        
        o, _ = env_base.reset(states=state_list[i], state_only=True, base=True)
        ep_ret, ep_len = 0, 0
        for t in range(trj_len):
            a = ac_base.step(o)
            next_o, r, s = env_base.step(a)
            ep_ret += r
            ep_len += 1
            scr_buf[..., t] = s
            for e in range(test_ensemble_num):
                freq = np.unique(a[e], axis=0)
                unq_buf[e][t] = freq.shape[0]
            o = next_o

        baseline_data['Ret'].append(np.mean(ep_ret / ep_len))
        baseline_data['FinalScore'].append(np.mean(s))
        baseline_data['scr_buf'].append(scr_buf)
        baseline_data['unq_buf'].append(unq_buf)
    baseline_data['Ret'] = np.mean(baseline_data['Ret'])
    baseline_data['FinalScore'] = np.mean(baseline_data['FinalScore'])
    baseline_data['scr_buf'] = np.array(baseline_data['scr_buf'])
    baseline_data['unq_buf'] = np.array(baseline_data['unq_buf'])
    baseline_data_dict[baseline_name] = baseline_data
    print("Baseline finished")
    
with open('baseline_maxmc_N15K7NN3L200.pkl', 'wb') as f:
    pickle.dump(baseline_data_dict, f, pickle.HIGHEST_PROTOCOL)

In [4]:
env_num = 10
test_ensemble_num = 100
env_list = [envs.__dict__[env_name](**env_kwargs) for i in range(env_num)]
state_list = []
for i in range(env_num):
    _, fixed_state = env_list[i].reset(E=test_ensemble_num, base=True)
    state_list.append(deepcopy(fixed_state))
print("Baseline construction initiated")

Baseline construction initiated


In [5]:
import imp
imp.reload(core)
# Complete에서 볼 것 
# S2 전략이 50에서 제일 좋다는 것
# Max

baseline_data_dict = {}
baseline_data_dict['keys'] = ['Ret', 'FinalScore']

#names_list = [['30', '50', '70'], ['E','S'], ['T', 'F'], ['', '50', '80']]
names_list = [['10'], ['S'], ['F'], ['20', '40', '60', '80', '100']]
name_combination = (list(product(*names_list)))
mod_type_list = [''.join(name_combination[i]) for i in range(len(name_combination))]
#mod_type_list = ['50SF', '30SF', '70SF', '50SF80', '50EF', '30EF', '70EF', '50EF80']
print(mod_type_list)
baseline_name = 'RL_Inspired_SLSs'

for mod_name in mod_type_list:
    print(f"Baseline : {baseline_name}_{mod_name}")
    baseline_data = {}
    baseline_data['Ret'] = []
    baseline_data['FinalScore'] = []
    baseline_data['scr_buf'] = []
    baseline_data['unq_buf'] = []
    
    for i in range(env_num):
        print(i)
        env_base = env_list[i]
        ac_base = core.__dict__[baseline_name](env_base, action_type, extra_type, corr_type='TT', mod_type=mod_name)
        scr_buf = np.zeros((test_ensemble_num, M, trj_len), dtype=np.float32)
        unq_buf = np.zeros((test_ensemble_num, trj_len), dtype=np.float32)
        
        o, _ = env_base.reset(states=state_list[i], state_only=True, base=True)
        ep_ret, ep_len = 0, 0
        for t in range(trj_len):
            a = ac_base.step(o)
            next_o, r, s = env_base.step(a)
            ep_ret += r
            ep_len += 1
            scr_buf[..., t] = s
            for e in range(test_ensemble_num):
                freq = np.unique(a[e], axis=0)
                unq_buf[e][t] = freq.shape[0]
            o = next_o

        baseline_data['Ret'].append(np.mean(ep_ret / ep_len))
        baseline_data['FinalScore'].append(np.mean(s))
        baseline_data['scr_buf'].append(scr_buf)
        baseline_data['unq_buf'].append(unq_buf)
    baseline_data['Ret'] = np.mean(baseline_data['Ret'])
    baseline_data['FinalScore'] = np.mean(baseline_data['FinalScore'])
    baseline_data['scr_buf'] = np.array(baseline_data['scr_buf'])
    baseline_data['unq_buf'] = np.array(baseline_data['unq_buf'])
    baseline_data_dict[mod_name] = baseline_data
    
print("Baseline finished")
with open('baseline_maxmc_tweak1_N15K7NN3L200_RL.pkl', 'wb') as f:
    pickle.dump(baseline_data_dict, f, pickle.HIGHEST_PROTOCOL)

['10SF20', '10SF40', '10SF60', '10SF80', '10SF100']
Baseline : RL_Inspired_SLSs_10SF20
0
1
2
3
4
5
6
7
8
9
Baseline : RL_Inspired_SLSs_10SF40
0
1
2
3
4
5
6
7
8
9
Baseline : RL_Inspired_SLSs_10SF60
0
1
2
3
4
5
6
7
8
9
Baseline : RL_Inspired_SLSs_10SF80
0
1
2
3
4
5
6
7
8
9
Baseline : RL_Inspired_SLSs_10SF100
0
1
2
3
4
5
6
7
8
9
Baseline finished


In [ ]:
import imp
imp.reload(core)

#names_list = [['30', '50', '70'], ['E','S'], ['T', 'F'], ['', '50', '80']]
#names_list = [['30', '50', '70'], ['S'], ['F'], ['', '50', '80']]
#name_combination = (list(product(*names_list)))
#mod_type_list = [''.join(name_combination[i]) for i in range(len(name_combination))]
mod_type_list = ['50SF80', '50SF60', '50SF', '50EF80', '50EF60', '50EF']
print(mod_type_list)
baseline_name = 'RL_Inspired_SLSs'

for mod_name in mod_type_list:
    print(f"Baseline : {baseline_name}_{mod_name}")
    baseline_data = {}
    baseline_data['Ret'] = []
    baseline_data['FinalScore'] = []
    baseline_data['scr_buf'] = []
    baseline_data['unq_buf'] = []
    
    for i in range(env_num):
        print(i)
        env_base = env_list[i]
        ac_base = core.__dict__[baseline_name](env_base, action_type, extra_type, corr_type='TT', mod_type=mod_name)
        scr_buf = np.zeros((test_ensemble_num, M, trj_len), dtype=np.float32)
        unq_buf = np.zeros((test_ensemble_num, trj_len), dtype=np.float32)
        
        o, _ = env_base.reset(states=state_list[i], state_only=True, base=True)
        ep_ret, ep_len = 0, 0
        for t in range(trj_len):
            a = ac_base.step(o)
            next_o, r, s = env_base.step(a)
            ep_ret += r
            ep_len += 1
            scr_buf[..., t] = s
            for e in range(test_ensemble_num):
                freq = np.unique(a[e], axis=0)
                unq_buf[e][t] = freq.shape[0]
            o = next_o

        baseline_data['Ret'].append(np.mean(ep_ret / ep_len))
        baseline_data['FinalScore'].append(np.mean(s))
        baseline_data['scr_buf'].append(scr_buf)
        baseline_data['unq_buf'].append(unq_buf)
    baseline_data['Ret'] = np.mean(baseline_data['Ret'])
    baseline_data['FinalScore'] = np.mean(baseline_data['FinalScore'])
    baseline_data['scr_buf'] = np.array(baseline_data['scr_buf'])
    baseline_data['unq_buf'] = np.array(baseline_data['unq_buf'])
    baseline_data_dict[mod_name] = baseline_data
    
print("Baseline finished")
with open('baseline_maxmc_N15K7NN3L200_RL.pkl', 'wb') as f:
    pickle.dump(baseline_data_dict, f, pickle.HIGHEST_PROTOCOL)

In [ ]:
for key in baseline_data_dict.keys():
    if key != 'keys':
        print(key)
        print(baseline_data_dict[key]['Ret'])

In [ ]:
with open(f'./data/baseline_data/baseline_maxmc_N15K7NN3.pkl', 'rb') as f:
    baseline_data_dict2 = pickle.load(f)

In [ ]:
with open('baseline_maxmc_N15K7NN3L200_RL.pkl', 'rb') as f:
    baseline_data_dict = pickle.load(f)

In [ ]:
# Figure drawing (RL_Inspired_SLSs)
baselines = ['FollowBest', 'FollowBest_indv', 'FollowMajor', 'FollowMajor_indv', 'IndvLearning', 'RandomCopy']

fig = plt.figure(figsize=(4,4), dpi=150)
ax = fig.add_subplot(111)
color_list = ['green', 'blue','orangered', 'gold', 'purple', 'cyan', 'black']
counter=0
for baseline_name in baselines:
    x = baseline_data_dict2[baseline_name]['scr_buf']
    avg_pf = np.mean(x, axis=tuple(range(0, len(x.shape) - 1)))
    std_pf = np.std(x, axis=tuple(range(0, len(x.shape) - 1)))
    ax.plot(np.arange(x.shape[-1]), avg_pf, c=color_list[counter], label=baseline_name)
    #ax.fill_between(np.arange(x.shape[-1]), avg_pf-std_pf, avg_pf+std_pf, facecolor=color_list[counter], alpha=0.2)
    counter+=1
        
#names_list = [['30', '50', '70'], ['E','S'], ['T', 'F'], ['', '50', '80']]
names_list = [['50', '70'], ['S'], ['T', 'F'], ['50', '80']]

name_combination = (list(product(*names_list)))
mod_type_list = [''.join(name_combination[i]) for i in range(len(name_combination))]
mod_type_list = ['50SF80', '50SF60', '50SF']
#mod_type_list = ['45ST2', '20ST2', '70ST2', '45ST1']
baseline_name = 'RL_Inspired_SLSs'   

for i, mod_name in enumerate(mod_type_list):
    x = baseline_data_dict[mod_name]['scr_buf'][..., :200]
    avg_pf = np.mean(x, axis=tuple(range(0, len(x.shape) - 1)))
    std_pf = np.std(x, axis=tuple(range(0, len(x.shape) - 1)))
    #ls = '-' if i<4 else '--'
    ls = '--'
    ax.plot(np.arange(x.shape[-1]), avg_pf, label=mod_name, ls=ls)
    #ax.fill_between(np.arange(x.shape[-1]), avg_pf-std_pf, avg_pf+std_pf, facecolor='black', alpha=0.2)
    counter+=1

#x = buf_scr_list
#avg_pf = np.mean(x, axis=tuple(range(0, len(x.shape) - 1)))
#std_pf = np.std(x, axis=tuple(range(0, len(x.shape) - 1)))
#ax.plot(np.arange(x.shape[-1]), avg_pf, c=color_list[counter], label='RL_TT')
#ax.fill_between(np.arange(x.shape[-1]), avg_pf-std_pf, avg_pf+std_pf, facecolor=color_list[counter], alpha=0.2)
ax.set_xlabel('Time')
ax.set_ylabel('Average Performance')
ax.legend(loc=4, fontsize=6)

In [ ]:
# Figure drawing
fig = plt.figure(figsize=(4,4), dpi=150)
ax = fig.add_subplot(111)
color_list = ['green', 'blue','orangered', 'gold', 'purple', 'cyan', 'black']
counter=0
if baselines:
    for baseline_name in baselines:
        x = baseline_data_dict[baseline_name]['scr_buf']
        avg_pf = np.mean(x, axis=tuple(range(0, len(x.shape) - 1)))
        std_pf = np.std(x, axis=tuple(range(0, len(x.shape) - 1)))
        ax.plot(np.arange(x.shape[-1]), avg_pf, c=color_list[counter], label=baseline_name)
        #ax.fill_between(np.arange(x.shape[-1]), avg_pf-std_pf, avg_pf+std_pf, facecolor=color_list[counter], alpha=0.2)
        counter+=1

#x = buf_scr_list
#avg_pf = np.mean(x, axis=tuple(range(0, len(x.shape) - 1)))
#std_pf = np.std(x, axis=tuple(range(0, len(x.shape) - 1)))
#ax.plot(np.arange(x.shape[-1]), avg_pf, c=color_list[counter], label='RL_TT')
#ax.fill_between(np.arange(x.shape[-1]), avg_pf-std_pf, avg_pf+std_pf, facecolor=color_list[counter], alpha=0.2)
ax.set_xlabel('Time')
ax.set_ylabel('Unique states')
ax.legend()

In [ ]:
# Figure drawing
fig = plt.figure(figsize=(4,4), dpi=150)
ax = fig.add_subplot(111)
color_list = ['green', 'blue','orangered', 'gold', 'purple', 'cyan', 'black']
counter=0
if baselines:
    for baseline_name in baselines:
        x = baseline_data_dict[baseline_name]['scr_buf']
        avg_pf = np.mean(x, axis=tuple(range(0, len(x.shape) - 1)))
        std_pf = np.std(x, axis=tuple(range(0, len(x.shape) - 1)))
        ax.plot(np.arange(x.shape[-1]), avg_pf, c=color_list[counter], label=baseline_name)
        #ax.fill_between(np.arange(x.shape[-1]), avg_pf-std_pf, avg_pf+std_pf, facecolor=color_list[counter], alpha=0.2)
        counter+=1

#x = buf_scr_list
#avg_pf = np.mean(x, axis=tuple(range(0, len(x.shape) - 1)))
#std_pf = np.std(x, axis=tuple(range(0, len(x.shape) - 1)))
#ax.plot(np.arange(x.shape[-1]), avg_pf, c=color_list[counter], label='RL_TT')
#ax.fill_between(np.arange(x.shape[-1]), avg_pf-std_pf, avg_pf+std_pf, facecolor=color_list[counter], alpha=0.2)
ax.set_xlabel('Time')
ax.set_ylabel('Unique states')
ax.legend()

In [ ]:
with open('baseline_complete_N15K7NN9.pkl', 'wb') as f:
    pickle.dump(baseline_data_dict, f, pickle.HIGHEST_PROTOCOL)

In [ ]:
with open(f'./data/baseline_data/baseline_maxmc_N15K7NN3.pkl', 'rb') as f:
    baseline_data = pickle.load(f)

In [ ]:
with open(f'./data/baseline_data/baseline_maxmc_N15K7NN3.pkl', 'rb') as f:
    baseline_data = pickle.load(f)

In [ ]:
baseline_data['FollowMajor_indv']['unq_buf']

In [ ]:
with open('baseline_complete_N15K7NN3_new.pkl', 'wb') as f:
    pickle.dump(baseline_data_dict, f, pickle.HIGHEST_PROTOCOL)

In [ ]:
# Figure drawing
fig = plt.figure(figsize=(4,4), dpi=150)
ax = fig.add_subplot(111)
color_list = ['green', 'blue','orangered', 'gold', 'purple', 'cyan']
counter=0
if baselines:
    for baseline_name in baselines:
        x = baseline_data_dict[baseline_name]['scr_buf'][:20, :20]
        avg_pf = np.mean(x, axis=tuple(range(0, len(x.shape) - 1)))
        std_pf = np.std(x, axis=tuple(range(0, len(x.shape) - 1)))
        ax.plot(np.arange(x.shape[-1]), avg_pf, c=color_list[counter], label=baseline_name)
        #ax.fill_between(np.arange(x.shape[-1]), avg_pf-std_pf, avg_pf+std_pf, facecolor=color_list[counter], alpha=0.2)
        counter+=1

ax.set_xlabel('Time')
ax.set_ylabel('Average Performance')
ax.legend()

## Model Loading

In [ ]:
def load_model(exp_name, epoch):

    #rel_path = f'data/runs/ds_complete_indv_raw_random_SIR_N10K3NN3_new_rand/{exp_name}/{exp_name}_s42/'
    rel_path = f'data/runs/{exp_name}/{exp_name}_s42/'

    with open(rel_path + "config.json") as json_file:
        json_data = json.load(json_file)
    env_kwargs = json_data['env_kwargs']
    env_name = json_data['env_name']
    env_kwargs['graph'] = nx.complete_graph
    ac_kwargs = json_data['ac_kwargs']
    ac_kwargs['activation'] = nn.Tanh()
    arch = json_data['arch']
    trj_len = json_data['trj_len']
    gamma = json_data['gamma']
    lam = json_data['lam']
    epochs = json_data['epochs']
    seed = json_data['seed']
    ensemble_num = env_kwargs['E']
    agent_num = env_kwargs['M']
    env_scheduler_kwargs = {
            'local_rank': 0,
            'exp_name': exp_name,
            'N': env_kwargs['N'],
            'K': env_kwargs['K'],
            'exp': env_kwargs['exp'],
            'NGPU': 1, #'data_dir': 'D:\\OneDrive\\연구\\ML\\MyProject\\SocialNet\\SocialNet\\data\\runs\\ds_complete_indv_raw_random_SIR_N10K3NN3_new_rand'
        'data_dir': 'D:\\OneDrive\\연구\\ML\\MyProject\\SocialNet\\SocialNet\\data\\runs'

    }
    env_kwargs['env_scheduler'] = envs.__dict__['random_env_scheduler'](**env_scheduler_kwargs)
    json_data['corr_type'] = 'TT'
    env_kwargs['corr_type'] = 'TT'
    if len(env_kwargs['reward_type']) < 9:
        print('modify')
        env_kwargs['reward_type'] = env_kwargs['reward_type'] + '_full'
    torch.manual_seed(seed)
    np.random.seed(seed)
    env = envs.__dict__[env_name](**env_kwargs)
    action_type = env_kwargs['action_type']
    extra_type = env_kwargs['extra_type']
    extra_num = len(extra_type)
    # Instantiate environment
    if action_type == 'total':
        obs_dim = (env.neighbor_num + 1, env.N + extra_num)  # (3+1, 15+2)
        act_dim = env.action_space.n
        dim_len = env.N
    elif action_type == 'split':
        obs_dim = (env.neighbor_num + 1, 1 + extra_num)
        act_dim = (2,)
        dim_len = env.N
        
    checkpoint = torch.load(rel_path+f'pyt_save/model{epoch}.pth')
    ac = core.ActorCritic(obs_dim, act_dim, arch, **ac_kwargs)
    ac.pi.load_state_dict(checkpoint['pi'])
    ac.v.load_state_dict(checkpoint['v'])

    Parallel = DataParallel
    parallel_args = {
        'device_ids': list(range(1)),
        'output_device': 0
    } 

    ac.pi = Parallel(ac.pi, **parallel_args)
    ac.v = Parallel(ac.v, **parallel_args)
    ac.eval()
    return ac, obs_dim, act_dim, dim_len, gamma, lam, env_kwargs

In [ ]:
# complete_L200 2820
# complete_L300 2593
# maxmc_L100 4483
# maxmc_L200 3761
exp_name = 'st_maxmc_indv_raw_full_total_random_SI_TT_N15K7NN3_new_rand200'
epoch = 3761
ac, obs_dim, act_dim, dim_len, gamma, lam, env_kwargs = load_model(exp_name, epoch)

## Model Test

In [ ]:
buf_list = []
final_score_list = []
unq_buf_list = []
prob_buf_list = []
Ret_list = []
env_num = 10
env_list = [envs.__dict__[env_name](**env_kwargs) for i in range(env_num)]

for i in range(env_num):
    print(i)
    test_ensemble_num = 10
    buf = PPOBuffer(
        obs_dim, 
        act_dim, 
        test_ensemble_num, 
        env_kwargs['M'], 
        dim_len, 
        trj_len, 
        gamma, 
        lam, 
        split=True if env_kwargs['action_type'] == 'split' else False)
    unq_buf = np.zeros((test_ensemble_num, trj_len), dtype=np.float32)
    prob_buf = []
    
    env = env_list[i]
    o, _ = env.reset(test_ensemble_num) 
    ep_ret, ep_len = 0, 0
    best_ep_ret = -np.inf
    env.scores.flatten().max()
    
    for t in range(trj_len):
        a, v, logp, pi = ac.step(torch.as_tensor(o, dtype=torch.float32, device='cuda'), return_pi=True)
        prob_buf.append(pi.probs[..., 1].detach().cpu().numpy())
        
        next_o, r, s = env.step(a)
        ep_ret += r
        ep_len += 1
        #print(np.mean(r), np.mean(s))
        # save and log
        buf.store(o, a, r, v, s, logp)

        # Update obs (critical!)
        o = next_o
        epoch_ended = t == trj_len - 1
        for e in range(test_ensemble_num):
            freq = np.unique(a[e], axis=0)
            unq_buf[e][t] = freq.shape[0]
        
        if epoch_ended:
            a, v, logp, pi = ac.step(
                torch.as_tensor(o, dtype=torch.float32, device='cuda' ),
                return_pi=True
            )
            _, _, s = env.step(a)
            buf.finish_path(v)
            prob_buf.append(pi.probs[..., 1].detach().cpu().numpy())
            for e in range(test_ensemble_num):
                freq = np.unique(a[e], axis=0)
                unq_buf[e][t] = freq.shape[0]
    
    unq_buf_list.append(unq_buf)
    prob_buf_list.append(prob_buf)
    Ret=ep_ret / ep_len
    Ret_list.append(Ret)
    EpLen=ep_len
    FinalScore=np.mean(s)
    buf_list.append(buf)
    final_score_list.append(FinalScore)
    ep_ret, ep_len = 0, 0

unq_buf_list = np.array(unq_buf_list)
prob_buf_list = np.array(prob_buf_list)
Ret_list = np.array(Ret_list)

In [ ]:
np.mean(Ret)

In [ ]:
inspection_dict = {'buf_list': buf_list, 'unq_buf_list': unq_buf_list, 'prob_buf_list': prob_buf_list}
with open('./result/inspection_dict/buf_list_maxmc_N15K7NN3L200.pkl', 'wb') as f:
    pickle.dump(inspection_dict, f, pickle.HIGHEST_PROTOCOL)

In [ ]:
with open('./result/inspection_dict/buf_list_maxmc_N15K7NN3L200.pkl', 'rb') as f:
    inspection_dict = pickle.load(f)

In [ ]:
buf_list = inspection_dict['buf_list']

In [ ]:
buf_scr_list = []
for i in range(len(buf_list)):
    buf_scr_list.append(buf_list[i].scr_buf)
buf_scr_list = np.array(buf_scr_list)

In [ ]:
# Figure drawing
fig = plt.figure(figsize=(4,4), dpi=150)
ax = fig.add_subplot(111)
color_list = ['green', 'blue','orangered', 'gold', 'purple', 'cyan', 'black']
counter=0

x = buf_scr_list
avg_pf = np.mean(x, axis=tuple(range(0, len(x.shape) - 1)))
std_pf = np.std(x, axis=tuple(range(0, len(x.shape) - 1)))
ax.plot(np.arange(x.shape[-1]), avg_pf, c=color_list[counter], label='RL_TT')
#ax.fill_between(np.arange(x.shape[-1]), avg_pf-std_pf, avg_pf+std_pf, facecolor=color_list[counter], alpha=0.2)
ax.set_xlabel('Time')
ax.set_ylabel('Average Performance')
ax.legend()

In [ ]:
plt.imshow(buf_list[0].act_buf[0][:, 14, :]) # ensemble / agent / traj_len / N+2

In [ ]:
plt.imshow(buf_list[0].obs_buf[0][:, 14, 0, :-2]) # ensemble / agent / traj_len / neighbor+1 / N+2

In [ ]:
# Figure drawing
fig = plt.figure(figsize=(4,4), dpi=150)
ax = fig.add_subplot(111)


for i in range(100):
    ax.plot(buf_list[0].scr_buf[0][i])

ax.set_xlabel('Time')
ax.set_ylabel('Score')

In [ ]:
with open(f'./data/baseline_data/baseline_maxmc_N15K7NN3.pkl', 'rb') as f:
    baseline_data_dict = pickle.load(f)

In [ ]:
buf_list[0].act_buf[0][2][3]

In [ ]:
baselines = ['FollowBest', 'FollowBest_indv', 'FollowMajor', 'FollowMajor_indv', 'RandomCopy']

In [ ]:
# Figure drawing
fig = plt.figure(figsize=(2,2), dpi=150)
ax = fig.add_subplot(111)
color_list = ['green', 'blue','orangered', 'gold', 'purple', 'cyan']
counter=0
if baselines:
    for baseline_name in baselines:
        x = baseline_data_dict[baseline_name]['scr_buf']
        avg_pf = np.mean(x, axis=tuple(range(0, len(x.shape) - 1)))
        std_pf = np.std(x, axis=tuple(range(0, len(x.shape) - 1)))
        ax.plot(np.arange(x.shape[-1]), avg_pf, c=color_list[counter], label=baseline_name)
        #ax.fill_between(np.arange(x.shape[-1]), avg_pf-std_pf, avg_pf+std_pf, facecolor=color_list[counter], alpha=0.2)
        counter+=1

x = buf_scr_list
avg_pf = np.mean(x, axis=tuple(range(0, len(x.shape) - 1)))
std_pf = np.std(x, axis=tuple(range(0, len(x.shape) - 1)))
ax.plot(np.arange(x.shape[-1]), avg_pf, c='k', lw=3, label='RL_TT')

#ax.fill_between(np.arange(x.shape[-1]), avg_pf-std_pf, avg_pf+std_pf, facecolor=color_list[counter], alpha=0.2)
ax.set_xlabel('Time')
ax.set_ylabel('Average Performance')
ax.legend(fontsize=4, loc=4)

In [ ]:
# Figure drawing
fig = plt.figure(figsize=(4,4), dpi=150)
ax = fig.add_subplot(111)
color_list = ['green', 'blue','orangered', 'gold', 'purple', 'cyan', 'black']
counter=0

x = unq_buf_list
avg_pf = np.mean(x, axis=tuple(range(0, len(x.shape) - 1)))
std_pf = np.std(x, axis=tuple(range(0, len(x.shape) - 1)))
ax.plot(np.arange(x.shape[-1]), avg_pf, c='black', lw=3, label='RL_TT')

x = baseline_data['FollowMajor_indv']['unq_buf']
avg_pf = np.mean(x, axis=tuple(range(0, len(x.shape) - 1)))
std_pf = np.std(x, axis=tuple(range(0, len(x.shape) - 1)))
ax.plot(np.arange(x.shape[-1]), avg_pf, c=color_list[counter], label='FollowMajor_indv')
#ax.fill_between(np.arange(x.shape[-1]), avg_pf-std_pf, avg_pf+std_pf, facecolor=color_list[counter], alpha=0.2)
counter+=1


#ax.fill_between(np.arange(x.shape[-1]), avg_pf-std_pf, avg_pf+std_pf, facecolor=color_list[counter], alpha=0.2)
ax.set_xlabel('Time')
ax.set_ylabel('Unique states')
ax.legend()

In [ ]:
# Figure drawing
fig = plt.figure(figsize=(4,4), dpi=150)
ax = fig.add_subplot(111)
color_list = ['green', 'blue','orangered', 'gold', 'purple', 'cyan', 'black']
counter=0

x = buf_scr_list
avg_pf = np.mean(x, axis=tuple(range(0, len(x.shape) - 1)))
std_pf = np.std(x, axis=tuple(range(0, len(x.shape) - 1)))
ax.plot(np.arange(x.shape[-1]), avg_pf, c='black', lw=3, label='RL_TT')
#ax.fill_between(np.arange(x.shape[-1]), avg_pf-std_pf, avg_pf+std_pf, facecolor=color_list[counter], alpha=0.2)


if baselines:
    for baseline_name in baselines:
        x = baseline_data_dict[baseline_name]['scr_buf']
        avg_pf = np.mean(x, axis=tuple(range(0, len(x.shape) - 1)))
        std_pf = np.std(x, axis=tuple(range(0, len(x.shape) - 1)))
        ax.plot(np.arange(x.shape[-1]), avg_pf, c=color_list[counter], label=baseline_name)
        #ax.fill_between(np.arange(x.shape[-1]), avg_pf-std_pf, avg_pf+std_pf, facecolor=color_list[counter], alpha=0.2)
        counter+=1


ax.set_xlabel('Time')
ax.set_ylabel('Average Performance')
ax.legend(fontsize=6)


## Inspection

In [ ]:
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import matplotlib.gridspec as gridspec

In [ ]:
@interact
def inspect(i=widgets.IntSlider(min=0, max=env_num-1, step=1, value=0, description='Environment: '), 
            j=widgets.IntSlider(min=0, max=ensemble_num-1, step=1, value=0, description='Ensemble: '),
            k=widgets.IntSlider(min=0, max=agent_num-1, step=1, value=0, description='Agent: '),
            t=widgets.IntSlider(min=0, max=trj_len-1, step=1, value=0, description='Time:'),
           ):
    fig = plt.figure(figsize = (12, 4), dpi=200, constrained_layout=True)
    widths = [0.5, 0.2, 0.2, 0.2]
    heights = [3, 1]
    gs = fig.add_gridspec(nrows=2, ncols=4, width_ratios=widths, height_ratios=heights)

    ax00 = fig.add_subplot(gs[0, 0])
    indv_scr_data = buf_list[i].scr_buf[j][k]
    ax00.plot(indv_scr_data, c='k')
    ax00.plot(t, indv_scr_data[t], marker='o', ms=5, c='r')
    ax00.set_ylim(0, 100)

    ax01 = fig.add_subplot(gs[0, 1])
    prob_data = prob_buf_list[i][t, j, :]
    ax01.imshow(prob_data, cmap=cm.binary, aspect=0.3)
    ax01.set_title('Prob.')

    ax02 = fig.add_subplot(gs[0, 2])
    act_data = buf_list[i].act_buf[j,:, t]
    ax02.imshow(act_data, cmap=cm.binary, aspect=0.3)
    ax02.set_title('Action')

    ax03 = fig.add_subplot(gs[0, 3])
    next_state_data = buf_list[i].obs_buf[j,:, t+1, 0, :-2]
    ax03.imshow(next_state_data, cmap=cm.binary, aspect=0.3)
    next_state_score = env_list[0].get_score(next_state_data.reshape(1, agent_num, env_kwargs['N']))
    ax03.set_title(f'State (max : {np.max(next_state_score):.2f}, {np.argmax(next_state_score)}))')
    ax00.plot(buf_list[i].scr_buf[j][np.argmax(next_state_score)], c='r', alpha=0.5)

    ax10 = fig.add_subplot(gs[1, 0])
    indv_obs_data = buf_list[i].obs_buf[j, k, t, :, :-2]
    indv_obs_score = env_list[0].get_score(indv_obs_data.reshape(1, env_kwargs['neighbor_num']+1, env_kwargs['N'])).squeeze()
    ax10.imshow(indv_obs_data, cmap=cm.binary)
    ax10.set_title(f'Obs. (T={t}, Agent No.{k})')
    #ax10.set_xticks([])
    #ax10.set_yticks([])

    ax10.annotate("score", (15, -0.5), fontsize=10, annotation_clip=False)
    ax10.annotate("%0.2f" % indv_obs_score[0], (15, 0.5), fontsize=10, annotation_clip=False)
    ax10.annotate("%0.2f" % indv_obs_score[1], (15, 1.5), fontsize=10, annotation_clip=False)
    ax10.annotate("%0.2f" % indv_obs_score[2], (15, 2.5), fontsize=10, annotation_clip=False)
    ax10.annotate("%0.2f" % indv_obs_score[3], (15, 3.5), fontsize=10, annotation_clip=False)

    ax11 = fig.add_subplot(gs[1, 1])
    prob_data = prob_buf_list[i][t, j, k].reshape(1, -1)
    ax11.imshow(prob_data, cmap=cm.binary)
    ax11.set_title('Prob.')
    ax11.set_xticks([])
    ax11.set_yticks([])

    ax12 = fig.add_subplot(gs[1, 2])
    act_data = buf_list[i].act_buf[j, k, t].reshape(1, -1)
    ax12.imshow(act_data, cmap=cm.binary)
    action_score = env_list[0].get_score(act_data.reshape(1, 1, -1)).item()
    ax12.annotate("%0.2f" % action_score, (5, 2), fontsize=10, annotation_clip=False)
    ax12.set_title('Action')
    ax12.set_xticks([])
    ax12.set_yticks([])

    ax13 = fig.add_subplot(gs[1, 3])
    next_state_data = buf_list[i].obs_buf[j,k, t+1, 0, :-2].reshape(1, -1)
    ax13.imshow(next_state_data, cmap=cm.binary)
    next_state_score = env_list[0].get_score(next_state_data.reshape(1, 1, -1)).item()
    ax13.annotate("%0.2f" % next_state_score, (5, 2), fontsize=10, annotation_clip=False)
    ax13.set_title(f'State (T={t+1})')
    ax13.set_xticks([])
    ax13.set_yticks([])

In [ ]:
@interact
def operation(s1=widgets.IntSlider(min=0, max=100, step=1, value=15, description='Score_1:'), 
            s2=widgets.IntSlider(min=0, max=100, step=1, value=10, description='Score_2:'),
            s3=widgets.IntSlider(min=0, max=100, step=1, value=5, description='Score_3:'),
            s4=widgets.IntSlider(min=0, max=100, step=1, value=20, description='Score_4:')
             ):
    fig = plt.figure(figsize = (12, 4), dpi=200, constrained_layout=True)
    widths = [0.5, 0.2, 0.2, 0.2]
    heights = [1]
    gs = fig.add_gridspec(nrows=1, ncols=4, width_ratios=widths, height_ratios=heights)

    ax10 = fig.add_subplot(gs[0])
    
    #'''
    indv_obs_data = np.array([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, s1, 1], 
              [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, s2, 0],
                [1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, s3, 0],
              [0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, s4, 0]])
    '''
    indv_obs_data = np.array([[1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, s1, 1], 
              [1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, s2, 0],
                [1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, s3, 0],
              [0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, s4, 0]])
    '''
    indv_obs_score = [s1, s2, s3, s4]
    ax10.imshow(indv_obs_data[..., :-2], cmap=cm.binary)
    ax10.set_title(f'Observation')
    #ax10.set_xticks([])
    #ax10.set_yticks([])

    ax10.annotate("score", (15, -0.5), fontsize=10, annotation_clip=False)
    ax10.annotate("%0.2f" % indv_obs_score[0], (15, 0.5), fontsize=10, annotation_clip=False)
    ax10.annotate("%0.2f" % indv_obs_score[1], (15, 1.5), fontsize=10, annotation_clip=False)
    ax10.annotate("%0.2f" % indv_obs_score[2], (15, 2.5), fontsize=10, annotation_clip=False)
    ax10.annotate("%0.2f" % indv_obs_score[3], (15, 3.5), fontsize=10, annotation_clip=False)

    ax11 = fig.add_subplot(gs[1])
    a, v, logp, pi = ac.step(torch.as_tensor(indv_obs_data, dtype=torch.float32, device='cuda').unsqueeze(0), pi=True)
    prob_data = pi.probs[..., 1].detach().cpu().numpy()
    ax11.imshow(prob_data, cmap=cm.binary, vmin=0, vmax=1)
    ax11.set_title('Prob.')
    ax11.set_xticks([])
    ax11.set_yticks([])

    ax12 = fig.add_subplot(gs[2])
    act_data = a
    ax12.imshow(act_data, cmap=cm.binary)
    action_score = env_list[0].get_score(act_data.reshape(1, 1, -1)).item()
    ax12.set_title('Action')
    ax12.set_xticks([])
    ax12.set_yticks([])

# 1등 점수, 2등과 3등 점수의 합 (대충?), 2등과 1등의 점수 차가 영향을 미침
# 내 점수가 20 이상인데 다른 개체들과의 차이가 심하면 (최저 -10?) 메이저 베낌
# 자기보다 다들 높으면 확실히 best로 가려고 함 (2등과 1등 점수차가 상대적으로 작아도)
# 반대로 2, 3등이 고만고만하면 1등과의 점수차가 압도적이어야 바꿈
# 자기 점수가 높을 수록 2등, 3등 점수와 1등 점수차도 더 벌어짐 (+상수보다는 선형?)
# 자기가 꼴찌가 아니면 전략 안 바꿈! (하지만 극단적으로 높은 점수는 가끔 받음)

In [ ]:
@interact
def test1(s1=widgets.IntSlider(min=0, max=100, step=1, value=15, description='Score_1:'), 
            s2=widgets.IntSlider(min=0, max=100, step=1, value=10, description='Score_2:'),
            s3=widgets.IntSlider(min=0, max=100, step=1, value=5, description='Score_3:'),
            s4=widgets.IntSlider(min=0, max=100, step=1, value=20, description='Score_4:')
             ):
    %matplotlib inline
    fig = plt.figure(figsize = (12, 4), dpi=200, constrained_layout=True)
    widths = [0.5, 0.2, 0.2, 0.2]
    heights = [1]
    gs = fig.add_gridspec(nrows=1, ncols=4, width_ratios=widths, height_ratios=heights)

    ax10 = fig.add_subplot(gs[0])
    
    #'''
    indv_obs_data = np.array([
                  [1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, s1, 1],
                [0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, s2, 0],
                    [0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, s3, 0],
                  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, s4, 0]])
    '''
    indv_obs_data = np.array([[1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, s1, 1], 
              [1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, s2, 0],
                [1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, s3, 0],
              [0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, s4, 0]])
    '''
    indv_obs_score = [s1, s2, s3, s4]
    ax10.imshow(indv_obs_data[..., :-2], cmap=cm.binary)
    ax10.set_title(f'Observation')
    #ax10.set_xticks([])
    #ax10.set_yticks([])

    ax10.annotate("score", (15, -0.5), fontsize=10, annotation_clip=False)
    ax10.annotate("%0.2f" % indv_obs_score[0], (15, 0.5), fontsize=10, annotation_clip=False)
    ax10.annotate("%0.2f" % indv_obs_score[1], (15, 1.5), fontsize=10, annotation_clip=False)
    ax10.annotate("%0.2f" % indv_obs_score[2], (15, 2.5), fontsize=10, annotation_clip=False)
    ax10.annotate("%0.2f" % indv_obs_score[3], (15, 3.5), fontsize=10, annotation_clip=False)

    ax11 = fig.add_subplot(gs[1])
    a, v, logp, pi = ac.step(torch.as_tensor(indv_obs_data, dtype=torch.float32, device='cuda').unsqueeze(0), return_pi=True)
    prob_data = pi.probs[..., 1].detach().cpu().numpy()
    ax11.imshow(prob_data, cmap=cm.binary, vmin=0, vmax=1)
    ax11.set_title('Prob.')
    ax11.set_xticks([])
    ax11.set_yticks([])

    ax12 = fig.add_subplot(gs[2])
    act_data = a
    ax12.imshow(act_data, cmap=cm.binary)
    #action_score = env_list[0].get_score(act_data.reshape(1, 1, -1)).item()
    ax12.set_title('Action')
    ax12.set_xticks([])
    ax12.set_yticks([])

# 1등 점수, 2등과 3등 점수의 합 (대충?), 2등과 1등의 점수 차가 영향을 미침
# 내 점수가 20 이상인데 다른 개체들과의 차이가 심하면 (최저 -10?) 메이저 베낌
# 자기보다 다들 높으면 확실히 best로 가려고 함 (2등과 1등 점수차가 상대적으로 작아도)
# 반대로 2, 3등이 고만고만하면 1등과의 점수차가 압도적이어야 바꿈
# 자기 점수가 높을 수록 2등, 3등 점수와 1등 점수차도 더 벌어짐 (+상수보다는 선형?)
# 자기가 꼴찌가 아니면 전략 안 바꿈! (하지만 극단적으로 높은 점수는 가끔 받음)

In [ ]:
interact(inspection, buf_list=fixed(buf_list), )

In [ ]:
# Figure drawing
fig = plt.figure(figsize=(4,4), dpi=150)
ax = fig.add_subplot(111)
color_list = ['green', 'blue','orangered', 'gold', 'purple', 'cyan', 'black']
counter=0
if baselines:
    baseline_name = 'FollowMajor_indv'
    x = baseline_data[baseline_name]['scr_buf'][:20, :20]
    avg_pf = np.mean(x, axis=tuple(range(0, len(x.shape) - 1)))
    std_pf = np.std(x, axis=tuple(range(0, len(x.shape) - 1)))
    ax.plot(np.arange(x.shape[-1]), avg_pf, c=color_list[counter], label=baseline_name)
    #ax.fill_between(np.arange(x.shape[-1]), avg_pf-std_pf, avg_pf+std_pf, facecolor=color_list[counter], alpha=0.2)
    counter+=1


#x = buf_scr_list
#avg_pf = np.mean(x, axis=tuple(range(0, len(x.shape) - 1)))
#std_pf = np.std(x, axis=tuple(range(0, len(x.shape) - 1)))
#ax.plot(np.arange(x.shape[-1]), avg_pf, c=color_list[counter], label='RL_TT')
#ax.fill_between(np.arange(x.shape[-1]), avg_pf-std_pf, avg_pf+std_pf, facecolor=color_list[counter], alpha=0.2)
ax.set_xlabel('Time')
ax.set_ylabel('Average Performance')
ax.legend()

In [ ]:
# action
baselines = ['FollowBest', 'FollowBest_indv', 'FollowMajor', 'FollowMajor_indv', 'IndvLearning', 'RandomCopy']
insp_dict = {}
insp_dict['RL'] = [[] for i in range(env_num)]
for baseline_name in baselines:
    insp_dict[baseline_name] = [[] for i in range(env_num)]

for i in range(env_num):
    print(i)
    env_base = env_list[i]
    for j in range(trj_len):
        o = buf_list[i].obs_buf[:, :, j]
        insp_dict['RL'][i].append((ac.pi.module._distribution(torch.as_tensor(o, dtype=torch.float32, device='cuda')).probs[..., 1]).detach().cpu().numpy())
        for baseline_name in baselines:
            insp_dict[baseline_name][i].append(core.__dict__[baseline_name](env_base, action_type, extra_type).step(o))

In [ ]:
fig = plt.figure(figsize=(4,4), dpi=150)
ax = fig.add_subplot(111)
ax.imshow(buf_list[0].obs_buf[0][:, 50, 0, :-2])

In [ ]:
fig = plt.figure(figsize=(4,4), dpi=150)
ax = fig.add_subplot(111)
plt.imshow(buf_list[0].act_buf[0][70, :, :-2])

In [ ]:
insp_dict['RL'] = np.array(insp_dict['RL'])
insp_dict['RL_flatten'] = (insp_dict['RL']>=0.5).astype(np.long)
for baseline_name in baselines:
    insp_dict[baseline_name] = np.array(insp_dict[baseline_name])

In [ ]:
# dist

rl2b_dist = np.sqrt(np.sum((insp_dict['RL_flatten'] - insp_dict['FollowBest_indv'])**2, axis=-1))
rl2m_dist = np.sqrt(np.sum((insp_dict['RL_flatten'] - insp_dict['FollowMajor_indv'])**2, axis=-1))
rl2i_dist = np.sqrt(np.sum((insp_dict['RL_flatten'] - insp_dict['IndvLearning'])**2, axis=-1))
rl2r_dist = np.sqrt(np.sum((insp_dict['RL_flatten'] - insp_dict['RandomCopy'])**2, axis=-1))
m2b_dist = np.sqrt(np.sum((insp_dict['FollowBest_indv'] - insp_dict['FollowMajor_indv'])**2, axis=-1))

In [ ]:
fig = plt.figure(figsize=(4,4), dpi=150)
ax = fig.add_subplot(111)
i=4
ax.plot(np.mean(rl2b_dist[i], axis=(1, 2)), label = 'rl2b')
ax.plot(np.mean(rl2m_dist[i], axis=(1, 2)), label = 'rl2m')
ax.plot(np.mean(rl2i_dist[i], axis=(1, 2)), label = 'rl2i')
ax.plot(np.mean(rl2r_dist[i], axis=(1, 2)), label = 'rl2r')
ax.plot(np.mean(m2b_dist[i], axis=(1, 2)), label = 'm2b')
ax.legend()

In [ ]:
def matrix_cosine(x, y):
    return np.einsum('ij,ij->i', x, y) / (
              np.linalg.norm(x, axis=1) * np.linalg.norm(y, axis=1)
    )

In [ ]:
rl2b_cos = matrix_cosine(insp_dict['RL'].reshape(-1, 15), insp_dict['FollowBest_indv'].reshape(-1, 15)).reshape(5, 100, 5, 100)
rl2m_cos = matrix_cosine(insp_dict['RL'].reshape(-1, 15), insp_dict['FollowMajor_indv'].reshape(-1, 15)).reshape(5, 100, 5, 100)
rl2i_cos = matrix_cosine(insp_dict['RL'].reshape(-1, 15), insp_dict['IndvLearning'].reshape(-1, 15)).reshape(5, 100, 5, 100)
rl2r_cos = matrix_cosine(insp_dict['RL'].reshape(-1, 15), insp_dict['RandomCopy'].reshape(-1, 15)).reshape(5, 100, 5, 100)
m2b_cos = matrix_cosine(insp_dict['FollowMajor_indv'].reshape(-1, 15), insp_dict['FollowBest_indv'].reshape(-1, 15)).reshape(5, 100, 5, 100)

In [ ]:
fig = plt.figure(figsize=(4,4), dpi=150)
ax = fig.add_subplot(111)
i=4
ax.plot(1 - np.mean(rl2b_cos[i], axis=(1, 2)), label = 'rl2b')
ax.plot(1 - np.mean(rl2m_cos[i], axis=(1, 2)), label = 'rl2m')
ax.plot(1 - np.mean(rl2i_cos[i], axis=(1, 2)), label = 'rl2i')
ax.plot(1 - np.mean(rl2r_cos[i], axis=(1, 2)), label = 'rl2r')
ax.plot(1 - np.mean(m2b_cos[i], axis=(1, 2)), label = 'm2b')
ax.legend()

In [ ]:
# result
baselines = ['FollowBest', 'FollowBest_indv', 'FollowMajor', 'FollowMajor_indv', 'IndvLearning', 'RandomCopy']
insp_dict = {}
insp_dict['RL'] = [[] for i in range(env_num)]
for baseline_name in baselines:
    insp_dict[baseline_name] = [[] for i in range(env_num)]

for i in range(env_num):
    print(i)
    env_base = env_list[i]
    for j in range(trj_len):
        o = buf_list[i].obs_buf[:, :, j]
        a, v, logp = ac.step(torch.as_tensor(o, dtype=torch.float32, device='cuda'))
        next_o, r, s = env_base.step(a)
        insp_dict['RL'][i].append(next_o[:, :, 0, :-2])
        for baseline_name in baselines:
            insp_dict[baseline_name][i].append(core.__dict__[baseline_name](env_base, action_type, extra_type).step(o))

In [ ]:
insp_dict['RL'] = np.array(insp_dict['RL'])
insp_dict['RL_flatten'] = (insp_dict['RL']>=0.5).astype(np.long)
for baseline_name in baselines:
    insp_dict[baseline_name] = np.array(insp_dict[baseline_name])
# dist

rl2b_dist = np.sqrt(np.sum((insp_dict['RL_flatten'] - insp_dict['FollowBest_indv'])**2, axis=-1))
rl2m_dist = np.sqrt(np.sum((insp_dict['RL_flatten'] - insp_dict['FollowMajor_indv'])**2, axis=-1))
rl2i_dist = np.sqrt(np.sum((insp_dict['RL_flatten'] - insp_dict['IndvLearning'])**2, axis=-1))
rl2r_dist = np.sqrt(np.sum((insp_dict['RL_flatten'] - insp_dict['RandomCopy'])**2, axis=-1))
m2b_dist = np.sqrt(np.sum((insp_dict['FollowBest_indv'] - insp_dict['FollowMajor_indv'])**2, axis=-1))

In [ ]:
fig = plt.figure(figsize=(4,4), dpi=150)
ax = fig.add_subplot(111)
i=4
ax.plot(np.mean(rl2b_dist[i], axis=(1, 2)), label = 'rl2b')
ax.plot(np.mean(rl2m_dist[i], axis=(1, 2)), label = 'rl2m')
ax.plot(np.mean(rl2i_dist[i], axis=(1, 2)), label = 'rl2i')
ax.plot(np.mean(rl2r_dist[i], axis=(1, 2)), label = 'rl2r')
ax.plot(np.mean(m2b_dist[i], axis=(1, 2)), label = 'm2b')
ax.legend()

In [ ]:
rl2b_cos = matrix_cosine(insp_dict['RL'].reshape(-1, 15), insp_dict['FollowBest_indv'].reshape(-1, 15)).reshape(5, 100, 5, 100)
rl2m_cos = matrix_cosine(insp_dict['RL'].reshape(-1, 15), insp_dict['FollowMajor_indv'].reshape(-1, 15)).reshape(5, 100, 5, 100)
rl2i_cos = matrix_cosine(insp_dict['RL'].reshape(-1, 15), insp_dict['IndvLearning'].reshape(-1, 15)).reshape(5, 100, 5, 100)
rl2r_cos = matrix_cosine(insp_dict['RL'].reshape(-1, 15), insp_dict['RandomCopy'].reshape(-1, 15)).reshape(5, 100, 5, 100)
m2b_cos = matrix_cosine(insp_dict['FollowMajor_indv'].reshape(-1, 15), insp_dict['FollowBest_indv'].reshape(-1, 15)).reshape(5, 100, 5, 100)

In [ ]:
fig = plt.figure(figsize=(4,4), dpi=150)
ax = fig.add_subplot(111)
i=4
ax.plot(1 - np.mean(rl2b_cos[i], axis=(1, 2)), label = 'rl2b')
ax.plot(1 - np.mean(rl2m_cos[i], axis=(1, 2)), label = 'rl2m')
ax.plot(1 - np.mean(rl2i_cos[i], axis=(1, 2)), label = 'rl2i')
ax.plot(1 - np.mean(rl2r_cos[i], axis=(1, 2)), label = 'rl2r')
ax.plot(1 - np.mean(m2b_cos[i], axis=(1, 2)), label = 'm2b')
ax.legend()

## Strategy

In [ ]:
import timeit

def coord_triplet(s):
    x = []  # np.zeros((int(s*(s+1)*(s+2)/6), 3))

    for i in range(s):
        for j in range(i, s):
            for k in range(j, s):
                x.append([i, j, k])
                
                
    return np.array(x)

def fixed_point(ac):
    data = np.array([[[1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 99, 1], 
                  [0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 99, 0],
                    [0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 99, 0],
                  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 99, 0]]])

    a, v, logp, pi = ac.step(torch.as_tensor(data, dtype=torch.float32, device='cuda'), return_pi=True)
    x = pi.probs[..., 1].detach().cpu().numpy()
    return np.round(x)[0].astype(np.int)

def explode(data):
    shape_arr = np.array(data.shape)
    size = shape_arr[:3]*2 - 1
    exploded = np.zeros(np.concatenate([size, shape_arr[3:]]), dtype=data.dtype)
    exploded[::2, ::2, ::2] = data
    return exploded

def expand_coordinates(indices):
    x, y, z = indices
    x[1::2, :, :] += 1
    y[:, 1::2, :] += 1
    z[:, :, 1::2] += 1
    return x, y, z

def make_cube(s, fp_dist, sp_dist, hp_dist):
    x = np.ones((s, s, s, 3))
    c = 0
    for i in range(s):
        for j in range(i, s):  # i+1
            for k in range(j, s):  # j+1
                x[i, j, k] = [fp_dist[c], sp_dist[c], hp_dist[c]]
                c+=1
    return x

def assign_facecolors(pi_list, fp, sp, hp, fp_show, sp_show, hp_show, max_s):
    fp_dist = (np.sum((pi_list-fp)**2, axis=-1)**0.5)/np.sqrt(N)
    sp_dist = (np.sum((pi_list-sp)**2, axis=-1)**0.5)/np.sqrt(N)
    hp_dist = (np.sum((pi_list-hp)**2, axis=-1)**0.5)/np.sqrt(N)
    
    facecolors = np.zeros((max_s, max_s, max_s, 4)) # R, G, B, alpha
    mc = make_cube(max_s, fp_dist, sp_dist, hp_dist)
    if fp_show:
        facecolors[..., 0] = 1 - mc[:, :, :, 0]   # Red : fp_dist
    if sp_show:
        facecolors[..., 1] = 1 - mc[:, :, :, 1]   # Red : fp_dist
    if hp_show:
        facecolors[..., 2] = 1 - mc[:, :, :, 2]   # Blue : hp_dist
    #facecolors[..., -1] = (np.maximum((1 - np.min(mc, axis=-1)), 0.5)-0.5)*2
    facecolors[..., -1] = (1 - np.min(mc[..., [fp_show, sp_show, hp_show]], axis=-1))**2 * 0.8 # maximum opacity : 0.8 * 0.8 
    return facecolors, fp_dist, sp_dist, hp_dist

def plot_cube(facecolors, stride, angle=320, name = '', save=True):
    IMG_DIM = len(facecolors)
    facecolors = explode(facecolors)
    
    filled = facecolors[:,:,:,-1] != 0
    #print(filled.shape)
    #print(np.indices(np.array(filled.shape) + 1).shape)
    x, y, z = expand_coordinates(np.indices(np.array(filled.shape) + 1))

    fig = plt.figure(figsize=(4, 4), dpi=200)
    ax = fig.add_subplot(111, projection='3d')
    ax.view_init(30, angle)
    ax.set_xlim(right=IMG_DIM*stride)
    ax.set_ylim(top=IMG_DIM*stride)
    ax.set_zlim(top=IMG_DIM*stride)
    
    ax.set_xlabel(r'$n_1$')
    ax.set_ylabel(r'$n_2$')
    ax.set_zlabel(r'$n_3 (highest)$')
    
    ax.voxels(x/2*stride, y/2*stride, z/2*stride, filled, facecolors=facecolors, shade=False)
    if save:
        plt.savefig(f'./result/cube_figure/cube_{name}.png', dpi=200)

In [ ]:
#%matplotlib notebook
%matplotlib inline
for s in range(5, 101, 5):  # 5, 10, ..., 95, 100
    print(s)
    max_s = 100
    '''
    data = np.array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -1, 1], 
                  [0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, -1, 0],
                    [1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, -1, 0],
                  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, -1, 0]])
    '''
    template = np.array([[1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -1, 1], 
                  [0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, -1, 0],
                    [0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, -1, 0],
                  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, -1, 0]])

    tc = coord_triplet(max_s)
    data = np.repeat(template.reshape(1, 4, 17), len(tc), axis=0)
    tc = np.c_[np.ones(tc.shape[0])*s, tc] 
    data[:, :, -2] = tc
    pi_list = []
    for i in range((data.shape[0]//10000)+1):
        a, v, logp, pi = ac.step(torch.as_tensor(data[i*10000:(i+1)*10000], dtype=torch.float32, device='cuda'), return_pi=True)
        x = pi.probs[..., 1].detach().cpu().numpy()
        pi_list.append(x)
    pi_list = np.concatenate(pi_list, axis=0)
    fp = fixed_point(ac)
    hp = template[-1][:-2]
    facecolors, fp_dist, hp_dist = assign_facecolors(pi_list, fp, hp, max_s)
    stride = 5
    assert max_s%stride == 0
    facecolors = facecolors[::stride, ::stride, ::stride]
    plot_cube(facecolors, stride = 5, angle=-75, name = f'maxmc_L200_E3761_S{s}')

In [ ]:
# complete_L200 2820
# complete_L300 2593
# maxmc_L100 4483
# maxmc_L200 3761
exp_name = 'st_maxmc_indv_raw_full_total_random_SI_TT_N15K7NN3_new_rand200'
epoch = 3761
ac, obs_dim, act_dim, dim_len, gamma, lam, env_kwargs = load_model(exp_name, epoch)

In [ ]:
%matplotlib notebook
s = 20
max_s = 100
'''
data = np.array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -1, 1], 
              [0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, -1, 0],
                [1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, -1, 0],
              [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, -1, 0]])
'''
template = np.array([[1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -1, 1], 
              [0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, -1, 0],
                [0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, -1, 0],
              [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, -1, 0]])

tc = coord_triplet(max_s)
data = np.repeat(template.reshape(1, 4, 17), len(tc), axis=0)
tc = np.c_[np.ones(tc.shape[0])*s, tc] 
data[:, :, -2] = tc
pi_list = []
for i in range((data.shape[0]//10000)+1):
    a, v, logp, pi = ac.step(torch.as_tensor(data[i*10000:(i+1)*10000], dtype=torch.float32, device='cuda'), return_pi=True)
    x = pi.probs[..., 1].detach().cpu().numpy()
    pi_list.append(x)
pi_list = np.concatenate(pi_list, axis=0)
fp = fixed_point(ac)
sp = template[0][:-2]
hp = template[-1][:-2]
facecolors, fp_dist, sp_dist, hp_dist = assign_facecolors(pi_list, fp, sp, hp, True, False, True, max_s)
stride = 5
assert max_s%stride == 0
facecolors = facecolors[::stride, ::stride, ::stride]
plot_cube(facecolors, stride = 5, angle=-75, name = str(s), save=True)

In [ ]:
s = 70
max_s = 100
'''
data = np.array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -1, 1], 
              [0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, -1, 0],
                [1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, -1, 0],
              [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, -1, 0]])
'''
template = np.array([[1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -1, 1], 
              [0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, -1, 0],
                [0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, -1, 0],
              [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, -1, 0]])

tc = coord_triplet(max_s)
data = np.repeat(template.reshape(1, 4, 17), len(tc), axis=0)
tc = np.c_[np.ones(tc.shape[0])*s, tc] 
data[:, :, -2] = tc
pi_list = []
for i in range((data.shape[0]//10000)+1):
    a, v, logp, pi = ac.step(torch.as_tensor(data[i*10000:(i+1)*10000], dtype=torch.float32, device='cuda'), return_pi=True)
    x = pi.probs[..., 1].detach().cpu().numpy()
    pi_list.append(x)
pi_list = np.concatenate(pi_list, axis=0)

In [ ]:
# baseline
mod_name = '50SF80'
baseline_name = 'RL_Inspired_SLSs'
ac_base = core.__dict__[baseline_name](env_base, action_type, extra_type, corr_type='TT', mod_type=mod_name)

%matplotlib notebook
s = 60
max_s = 100
'''
data = np.array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -1, 1], 
              [0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, -1, 0],
                [1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, -1, 0],
              [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, -1, 0]])
'''
template = np.array([[1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -1, 1], 
              [0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, -1, 0],
                [0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, -1, 0],
              [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, -1, 0]])

tc = coord_triplet(max_s)
data = np.repeat(template.reshape(1, 4, 17), len(tc), axis=0)
tc = np.c_[np.ones(tc.shape[0])*s, tc] 
data[:, :, -2] = tc
pi_list = []
for i in range((data.shape[0]//10000)+1):
    a = ac_base.step(data[i*10000:(i+1)*10000].reshape(2, -1, 4, 17))
    x = a.reshape(-1, 15)
    pi_list.append(x)
pi_list = np.concatenate(pi_list, axis=0)
fp = np.zeros(15)
sp = template[0][:-2]
hp = template[-1][:-2]
facecolors, fp_dist, sp_dist, hp_dist = assign_facecolors(pi_list, fp, sp, hp, False, True, True, max_s)
stride = 5
assert max_s%stride == 0
facecolors = facecolors[::stride, ::stride, ::stride]
plot_cube(facecolors, stride = 5, angle=-75, name = str(s), save=False)

In [ ]:
plt.imshow(pi_list, aspect='auto')
plt.colorbar()

## Supervised

In [ ]:
import envs
import ppo.core as core
from utils.utils import DataGen
from torch.utils.data import DataLoader
from torch.utils.data.dataset import TensorDataset
import ppo.net as net

In [ ]:
baseline_type = 'FollowBest'
exp_name = 'test'

batch_size = 16
batch_num = 2
total_size = batch_num * batch_size
train_ratio = 0.8

generator = DataGen(env, baseline_type, batch_size, batch_num)
if not os.path.isfile('./data/' + exp_name + '_train.pkl'):
    generator.run(exp_name, total_size, batch_size, train_ratio)

In [ ]:
with open('./data/' + exp_name + '_train.pkl', 'rb') as f:
    train_data = pickle.load(f)
with open('./data/' + exp_name + '_test.pkl', 'rb') as f:
    test_data = pickle.load(f)

In [ ]:
exp_name = 'complete_total_FollowBest_SIR_N10K3NN3'
with open('./data/supervised/' + exp_name + '_train.pkl', 'rb') as f:
    train_data = pickle.load(f)
with open('./data/supervised/' + exp_name + '_test.pkl', 'rb') as f:
    test_data = pickle.load(f)

In [ ]:
train_data_image = np.concatenate(train_data['Image'], axis=0)
train_data_image = train_data_image.reshape(-1, *train_data_image.shape[-2:])
train_data_label = np.concatenate(train_data['Label'], axis=0)
train_data_label = train_data_label.reshape(-1, *train_data_label.shape[-1:])
test_data_image = np.concatenate(test_data['Image'], axis=0)
test_data_image = test_data_image.reshape(-1, *test_data_image.shape[-2:])
test_data_label = np.concatenate(test_data['Label'], axis=0)
test_data_label = test_data_label.reshape(-1, *test_data_label.shape[-1:])

train_data = TensorDataset(torch.FloatTensor(train_data_image), torch.FloatTensor(train_data_label))
test_data = TensorDataset(torch.FloatTensor(test_data_image), torch.FloatTensor(test_data_label))

train_loader = DataLoader(
            train_data,
            batch_size=batch_size,
            shuffle=True,
            pin_memory=True,
            num_workers=0
        )
test_loader = DataLoader(
            test_data,
            batch_size=batch_size,
            shuffle=False,
            pin_memory=True,
            num_workers=0
        )

In [ ]:
model = net.__dict__['ds']((4, 13), 20).cuda()
exp_name = 'complete_total_FollowBest_SIR_N10K3NN3_CE_sptest'
checkpoint = torch.load(f'./data/runs/{exp_name}/{exp_name}_s42/pyt_save/model.pth')
model.load_state_dict(checkpoint)

# ETC

In [ ]:
def sample_gumbel(shape, eps=1e-20):
    U = torch.rand(shape).cuda()
    return -Variable(torch.log(-torch.log(U + eps) + eps))

def gumbel_softmax_sample(logits, temperature):
    y = logits + sample_gumbel(logits.size())
    return F.softmax(y / temperature, dim=-1)

def gumbel_softmax(logits, temperature):
    """
    input: [*, n_class]
    return: [*, n_class] an one-hot vector
    """
    y = gumbel_softmax_sample(logits, temperature)
    shape = y.size()
    _, ind = y.max(dim=-1)
    y_hard = torch.zeros_like(y).view(-1, shape[-1])
    y_hard.scatter_(1, ind.view(-1, 1), 1)
    y_hard = y_hard.view(*shape)
    return (y_hard - y).detach() + y

import math
print(gumbel_softmax(torch.cuda.FloatTensor([[math.log(0.1), math.log(0.4), math.log(0.3), math.log(0.2)]] * 20000), 1).sum(dim=0))

In [ ]:
def get_degree_preserving_randomization(edges):
    '''
    Randomizes a network provided by an edge list 
    producing neither self links nor duplicate links.
    The degree sequence will stay the same.
    INPUT:
    --- edges: list or set containing node pairs (tuples or lists of two nodes)
         
    OUTPUT:
    --- new_edges: new list containing new node pairs (tuples of two nodes)
    '''
    
    # make new set copy from edgelist
    edges = set( [tuple(e) for e in edges ]) 

    # get list of stubs
    stubs = [ ]
    [ stubs.extend(e) for e in edges ]

    # get a Counter object that counts the stubs for every node
    stub_counter = Counter(stubs)

    # initialize the new edge list
    new_edges = set()

    # get available nodes (nodes that have nonzero stub count)
    nodes = np.array([ stub for stub,count in stub_counter.items() if count!=0 ])

    # loop till the number of available nodes is zero
    while len(nodes)>0:

        # initialize dummy values for new edge
        first,second = -1,-1

        # choose edges that are not self-links (only possible if len(nodes)>1)
        while first == second and len(nodes)>1:
            first,second = np.random.choice(nodes,size=(2,),replace=False)

        # if the chosen (source,target) is are not the same
        # and not yet connected 
        # and there is more than one node with available stubs
        if first!=second and \
           (first,second) not in new_edges and \
           (second,first) not in new_edges and \
           len(nodes)>1:
            new_edges.add((first,second))
            stub_counter[first] -= 1
            stub_counter[second] -= 1
        else:
            # if not, pop a random edge and put its nodes 
            # back in the stub pool
            edge = random.sample(new_edges,1)[0]
            new_edges.remove(edge)
            stub_counter[edge[0]] += 1
            stub_counter[edge[1]] += 1

        # get available nodes (nodes that have nonzero stub count)
        nodes = np.array([ stub for stub,count in stub_counter.items() if count!=0 ])

        
    return list(new_edges)

In [ ]:
E = 32
M = 100
N = 15
K = 7
NN = 9
exp = 8
trj_len = 100
graph_type = 'complete'
reward_type = 'indv_raw'
action_type = 'total'
extra_type = 'SI'
env_name = 'SL_NK_' + action_type

nx_dict = {'complete': nx.complete_graph, 'ba': nx.barabasi_albert_graph, 'er': nx.erdos_renyi_graph} 
nx_arg_dict = {'complete': {'n': M}, 'ba': {'n': M, 'm': 19}, 'er': {'n': M, 'p': 0.3}}

env_kwargs = {
        'E': E,
        'M': M,
        'N': N,
        'K': K,
        'neighbor_num': NN,
        'exp': exp,
        'graph': nx_dict[graph_type],
        'graph_dict': nx_arg_dict[graph_type],
        'reward_type': reward_type,
        'action_type': action_type,
        'extra_type': extra_type,
    'corr_type': 'TT'
    }

In [ ]:
env_num = 1
test_ensemble_num = 1000
env_list = [envs.__dict__[env_name](**env_kwargs) for i in range(env_num)]
state_list = []
for i in range(env_num):
    _, fixed_state = env_list[i].reset(E=test_ensemble_num, base=True)
    state_list.append(deepcopy(fixed_state))
print("Baseline construction initiated")

In [ ]:
from ppo.core import Baseline
class FollowMajor_indv_test(Baseline):
    def __init__(self, env, action_type, extra_type, corr_type):
        super().__init__(env, action_type, extra_type, corr_type)
        print('test')
        self.landscape = env.landscape
        self.score_max = env.score_max

    def step(self, obs):
        with torch.no_grad():
            states_input = obs
            if self.action_type == 'total':
                E = obs.shape[0]
                M = obs.shape[1]
                N = obs.shape[3] - self.extra_num
                states = states_input[:, :, :, :N]
                states_neighbor = states[:, :, 1:, :]
                states = states[:, :, 0, :]
                scores = np.expand_dims(states_input[:, :, 0, N], axis=-1)
                scores_neighbor = states_input[:, :, 1:, N]
            elif self.action_type == 'split':
                E = obs.shape[0]
                M = obs.shape[1]
                N = obs.shape[2]
                states = states_input[:, :, :, :, 0]
                states_neighbor = states[:, :, :, 1:].transpose(0, 1, 3, 2)
                states = states[:, :, :, 0]
                scores = np.expand_dims(states_input[:, :, 0, 0, 1], axis=-1)
                scores_neighbor = states_input[:, :, 0, 1:, 1]
            else:
                raise NotImplementedError

            #states_social = np.copy(states)
            states_social = np.zeros_like(states)
            for i in range(E):
                for j in range(M):
                    states_unique, freq_states = np.unique(states_neighbor[i][j], axis=0, return_counts=True)
                    if len(freq_states) < self.env.neighbor_num:  # At least one 'most requent' state
                        #print(freq_states, freq_states.max(), freq_states==freq_states.max())
                        states_most_frequent = states_unique[freq_states == freq_states.max()]
                        
                        if len(states_most_frequent) == 1 :  # Single 'most frequent' state
                            states_social[i][j] = states_most_frequent[0]
                        else:  # Multiple 'most frequent' solutions
                            states_social[i][j] = states_most_frequent[np.random.randint(len(states_most_frequent))]
                    else:  # No frequent state
                        states_social[i][j] = states[i][j]
                        #print(states[i][j])
                        #indv_index = np.random.randint(N)
                        #states_social[i][j] = states[i][j]
                        #states_social[i][j][indv_index] = (states_social[i][j][indv_index] + 1) % 2
            '''
            scores_social = self.env.get_score(states=states_social)
            better_social = (scores_social > scores).astype(np.long)
            print(states_social, scores_social, better_social)
            
            '''
            print(states_social.shape)
            for e in range(test_ensemble_num):
                print(state_social[e])
                freq = np.unique(states_social[e], axis=0)
                print(freq)

            scores_social = self.env.get_score(states=states_social)
            better_social = (scores_social > scores).astype(np.long)
            
            index_indv = np.zeros_like(states)
            np.put_along_axis(index_indv, np.random.randint(0, N, (E, M, 1)), 1, axis=-1)
            states_indv = (states + index_indv) % 2
            scores_indv = self.env.get_score(states=states_indv)
            
            better_indv = (scores_indv > scores).astype(np.long) * (1 - better_social)  # not better social but better indv
            
            stay = (1 - better_social) * (1 - better_indv)
            if self.state_correction:
                states = (better_social * states_social) + (better_indv * states_indv) + stay * states
            else:
                states = states_social

            if self.reward_correction:
                scores = (scores_social * better_social) + (scores_indv * better_indv) + stay * scores
            else:
                scores = scores_social
                
            
            #states = better_social * states_social + (1 - better_social) * states
            #scores = better_social * scores_social + (1 - better_social) * scores
            #print(states, scores)

        return states

In [ ]:
baseline_data['act_buf'][0][0][:, :, 0][1]

In [ ]:
baseline_data['unq_buf'][0][0]

In [ ]:
a.shape

In [ ]:
baseline_data = {}
baseline_data['Ret'] = []
baseline_data['FinalScore'] = []
baseline_data['scr_buf'] = []
baseline_data['unq_buf'] = []
baseline_data['act_buf'] = []

for i in range(env_num):
    print(i)
    env_base = env_list[i]
    ac_base = FollowMajor_indv_test(env_base, action_type, extra_type, corr_type='TT')
    scr_buf = np.zeros((test_ensemble_num, M, trj_len), dtype=np.float32)
    unq_buf = np.zeros((test_ensemble_num, trj_len), dtype=np.float32)
    act_buf = np.zeros((test_ensemble_num, M, N, trj_len), dtype=np.float32)

    o, _ = env_base.reset(states=state_list[i], state_only=True, base=True)
    ep_ret, ep_len = 0, 0
    for t in range(2):
        print(t)
        a = ac_base.step(o)
        next_o, r, s = env_base.step(a)
        ep_ret += r
        ep_len += 1
        scr_buf[..., t] = s
        act_buf[..., t] = a
        for e in range(test_ensemble_num):
            freq = np.unique(a[e], axis=0)
            unq_buf[e][t] = freq.shape[0]
        o = next_o

    baseline_data['Ret'].append(np.mean(ep_ret / ep_len))
    baseline_data['FinalScore'].append(np.mean(s))
    baseline_data['scr_buf'].append(scr_buf)
    baseline_data['unq_buf'].append(unq_buf)
    baseline_data['act_buf'].append(act_buf)
baseline_data['Ret'] = np.mean(baseline_data['Ret'])
baseline_data['FinalScore'] = np.mean(baseline_data['FinalScore'])
baseline_data['scr_buf'] = np.array(baseline_data['scr_buf'])
baseline_data['unq_buf'] = np.array(baseline_data['unq_buf'])
baseline_data['act_buf'] = np.array(baseline_data['act_buf'])
print("Baseline finished")

In [ ]:
# Figure drawing
fig = plt.figure(figsize=(4,4), dpi=150)
ax = fig.add_subplot(111)
color_list = ['green', 'blue','orangered', 'gold', 'purple', 'cyan', 'black']
counter=0

x = baseline_data['unq_buf']
avg_pf = np.mean(x, axis=tuple(range(0, len(x.shape) - 1)))
std_pf = np.std(x, axis=tuple(range(0, len(x.shape) - 1)))
ax.plot(np.arange(x.shape[-1]), avg_pf, c=color_list[counter])
#ax.fill_between(np.arange(x.shape[-1]), avg_pf-std_pf, avg_pf+std_pf, facecolor=color_list[counter], alpha=0.2)
counter+=1

#x = buf_scr_list
#avg_pf = np.mean(x, axis=tuple(range(0, len(x.shape) - 1)))
#std_pf = np.std(x, axis=tuple(range(0, len(x.shape) - 1)))
#ax.plot(np.arange(x.shape[-1]), avg_pf, c=color_list[counter], label='RL_TT')
#ax.fill_between(np.arange(x.shape[-1]), avg_pf-std_pf, avg_pf+std_pf, facecolor=color_list[counter], alpha=0.2)
ax.set_xlabel('Time')
ax.set_ylabel('Unique states')
ax.legend()

In [ ]:
env_base = env_list[0]
ac_base = FollowMajor_indv_test(env_base, action_type, extra_type, corr_type='TT')

o = np.array([[[[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 3.83, 1], 
              [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 9.48, 0], 
              [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 9.48, 0],
              [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 9.48, 0],
              [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 9.48, 0],
              [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.21, 0],
              [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.21, 0],
              [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.21, 0], 
              [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.21, 0], 
              [1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 6.29, 0]]]])

'''
o = np.array([[[[1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 3.83, 1], 
              [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 5.13, 0],
                [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 5.13, 0],
              [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 10.15, 0]]]])

'''

for i in range(10):
    print(ac_base.step(o))

In [ ]:
from networkx.utils import nodes_or_number

In [ ]:
def maxmeanclustering(n):
    assert n%5==0
    s = int(n/5)
    A = 1 - np.eye(s)
    B = np.zeros((s, s))
    C = np.block([[A, B, B, B, B],
                 [B, A, B, B, B],
                 [B, B, A, B, B],
                 [B, B, B, A, B],
                 [B, B, B, B, A]])
    for i in range(5):
        j = s * i
        C[j][j+s-1] = 0
        C[j+s-1][j] = 0
        C[j][j-1] = 1
        C[j-1][j] = 1

    G = nx.from_numpy_matrix(C)
    return G